# CS4662 
## Group Project: Twitter Emotion Identification
### Instructor: Dr. Mohammad Porhomayoun

### Daniel Preciado

Spring 2020

In [1]:
# import modules
import json
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score

In [2]:

json_list = ['anger', 'fear', 'greed', 'hateful', 'joy', 'sadness']

all_list = []

for name in json_list:
    # load json data as a list of strings
    with open('raw_data/'+ name + '.json') as my_file:
        anger = json.load(my_file)
        
        # turn list to a list of tuples and append to all_list  
        for i in anger:
            all_list.append((i, name))
  

In [3]:
all_list[-1]

('Sad thing is sheep will buy into this POS https://t.co/hpeo1CSC70',
 'sadness')

In [4]:
# df = pd.DataFrame(np.array(list).reshape(-1,2), columns = ["comment", "emotion"])
df = pd.DataFrame(all_list, columns=['comment', 'label'])

In [5]:
# randomize the items inside dataframe
df = df.sample(frac=1).reset_index(drop=True)


In [6]:
df

,comment,label
0,RT @GlobalChamSIN: The global forum #Blockchai...,greed
1,@adams214 ur a dick adam sandham. Hope your ha...,joy
2,"Director, Data &amp; Analytics Strategy - NYA ...",greed
3,@CryptoCobain I might be missing something but...,fear
4,Trade bitcoin mining and cryptocurrency and ma...,greed
...,...,...
328357,"Bitcoin Price Jumps to 4-Month High Above $4,9...",greed
328358,Earning #cryptocurrency for selling my stuff o...,fear
328359,Pump result: #PIVX\n\n Start Price: 545\n We'...,greed
328360,Find more on #cancer https://t.co/5btZEGTnFz ...,greed


In [7]:
df.label.value_counts()

greed      200177
joy         54565
fear        50053
hateful     10206
sadness      9765
anger        3596
Name: label, dtype: int64

In [8]:
# define X and y
X = df.comment
y = df.label

In [9]:
from sklearn.model_selection import train_test_split

# Splitting the dataset into testing and training:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, 
                                                    random_state=6)

In [10]:
# examine the object shapes
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(229853,)
(98509,)
(229853,)
(98509,)


In [11]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()

In [12]:
# fit and transform X_train into X_train_dtm
X_train_dtm = vect.fit_transform(X_train)
X_train_dtm.shape

(229853, 312274)

In [13]:
# transform X_test into X_test_dtm
X_test_dtm = vect.transform(X_test)
X_test_dtm.shape

(98509, 312274)

In [14]:
# fit and transform X_train into X_train_dtm
y_train_dtm = vect.fit_transform(y_train)
y_train_dtm.shape


(229853, 6)

In [15]:
#adaboost classifier
from sklearn import metrics
from sklearn.ensemble import AdaBoostClassifier
my_AdaBoost = AdaBoostClassifier(n_estimators = 29,random_state=2)
my_AdaBoost.fit(X_train_dtm,y_train)
y_predict_prob_ab = my_AdaBoost.predict_proba(X_test_dtm)



y_predict_ab = my_AdaBoost.predict(X_test_dtm)
score_ab = accuracy_score(y_test, y_predict_ab)
print("Accuracy score adaboost ",score_ab)

Accuracy score adaboost  0.8173364870214904


In [16]:

from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(y_test, y_predict_ab))
cm = confusion_matrix(y_test, y_predict_ab)
print(cm)

              precision    recall  f1-score   support

       anger       0.81      0.08      0.15      1102
        fear       0.97      0.43      0.59     14953
       greed       0.79      0.98      0.87     60112
     hateful       0.72      0.68      0.70      3034
         joy       0.91      0.80      0.85     16402
     sadness       0.70      0.07      0.13      2906

    accuracy                           0.82     98509
   macro avg       0.82      0.51      0.55     98509
weighted avg       0.83      0.82      0.79     98509

[[   88     4   447   550    12     1]
 [    3  6372  8432    40    99     7]
 [    3   130 58696   175  1089    19]
 [    6     7   929  2059    28     5]
 [    7    52  3187     9 13088    59]
 [    1     6  2602    24    61   212]]


In [17]:
#xgboost classifier
from xgboost import XGBClassifier
my_XGBoost = XGBClassifier(n_estimators = 29,random_state=2)

my_XGBoost.fit(X_train_dtm,y_train)
y_predict_prob_xg = my_XGBoost.predict_proba(X_test_dtm)



y_predict_xg = my_XGBoost.predict(X_test_dtm)
score_xg = accuracy_score(y_test, y_predict_xg)
print("Accuracy score xgboost ",score_xg)

Accuracy score xgboost  0.9435990620146383


In [18]:
print(classification_report(y_test, y_predict_xg))
cm = confusion_matrix(y_test, y_predict_xg)
print(cm)

              precision    recall  f1-score   support

       anger       0.59      0.64      0.62      1102
        fear       0.94      0.93      0.93     14953
       greed       0.97      0.96      0.96     60112
     hateful       0.85      0.74      0.79      3034
         joy       0.92      0.97      0.94     16402
     sadness       0.85      0.84      0.84      2906

    accuracy                           0.94     98509
   macro avg       0.85      0.85      0.85     98509
weighted avg       0.94      0.94      0.94     98509

[[  709    13    81   277    18     4]
 [   13 13904   795    40   143    58]
 [   36   826 57715    75  1140   320]
 [  410    31   272  2254    43    24]
 [   17    41   359    10 15942    33]
 [   11    47   305    10   104  2429]]
